# Setup

In [ ]:
!pip install -q datasets
!pip install -q evaluate
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q accelerate -U

In [2]:
import copy
import datasets
import evaluate
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import warnings

from dataclasses import dataclass
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, AutoTokenizer, T5Config
from transformers import Trainer, TrainingArguments

In [3]:
tqdm.pandas()
warnings.filterwarnings("ignore", category=DeprecationWarning)

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
DEVICE

device(type='cuda')

In [4]:
MODEL_NAME = "t5-base"
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Prepare Dataset

In [5]:
df_train_java = datasets.load_dataset(
    "code_x_glue_ct_code_to_text",
    "java",
    split="train")
df_valid_java = datasets.load_dataset(
    "code_x_glue_ct_code_to_text",
    "java",
    split="validation")
df_test_java = datasets.load_dataset(
    "code_x_glue_ct_code_to_text",
    "java",
    split="test")

df_train_java = pd.DataFrame({
    'nl': df_train_java['docstring'],
    'code': df_train_java['code'],
    "language": "java"
})
df_valid_java = pd.DataFrame({
    'nl': df_valid_java['docstring'],
    'code': df_valid_java['code'],
    "language": "java"
})
df_test_java = pd.DataFrame({
    'nl': df_test_java['docstring'],
    'code': df_test_java['code'],
    "language": "java"
})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
df_train_go = datasets.load_dataset(
    "code_x_glue_ct_code_to_text",
    "go",
    split="train")
df_valid_go = datasets.load_dataset(
    "code_x_glue_ct_code_to_text",
    "go",
    split="validation")
df_test_go = datasets.load_dataset(
    "code_x_glue_ct_code_to_text",
    "go",
    split="test")

df_train_go = pd.DataFrame({
    'nl': df_train_go['docstring'],
    'code': df_train_go['code'],
    "language": "go"
})
df_valid_go = pd.DataFrame({
    'nl': df_valid_go['docstring'],
    'code': df_valid_go['code'],
    "language": "go"
})
df_test_go = pd.DataFrame({
    'nl': df_test_go['docstring'],
    'code': df_test_go['code'],
    "language": "go"
})

In [7]:
df_train_java.head()

,nl,code,language
0,Compare the supplied plaintext password to a h...,"public static boolean check(String passwd, Str...",java
1,Attempt to detect the current platform.\n\n@re...,public static Platform detect() throws Unsuppo...,java
2,Gets the node meta data.\n\n@param key - the m...,public <T> T getNodeMetaData(Object key) {\n ...,java
3,Copies all node meta data from the other node ...,public void copyNodeMetaData(ASTNode other) {\...,java
4,Sets the node meta data.\n\n@param key - the m...,"public void setNodeMetaData(Object key, Object...",java


In [8]:
df_train_go.head()

,nl,code,language
0,// getStringValue will return a quoted string ...,"func getStringValue(b []rune) (int, error) {\n...",go
1,// getBoolValue will return a boolean and the ...,"func getBoolValue(b []rune) (int, error) {\n\t...",go
2,// getNumericalValue will return a numerical s...,"func getNumericalValue(b []rune) (int, int, er...",go
3,// getNegativeNumber will return a negative nu...,func getNegativeNumber(b []rune) int {\n\tif b...,go
4,// isEscaped will return whether or not the ch...,"func isEscaped(value []rune, b rune) bool {\n\...",go


In [9]:
def add_java_slashes(a):
    return "// " + "\n// ".join(a.split("\n"))

In [10]:
df_train_java["nl"] = df_train_java["nl"].apply(add_java_slashes)
df_valid_java["nl"] = df_valid_java["nl"].apply(add_java_slashes)
df_test_java["nl"] = df_test_java["nl"].apply(add_java_slashes)
df_train_java.head()

,nl,code,language
0,// Compare the supplied plaintext password to ...,"public static boolean check(String passwd, Str...",java
1,// Attempt to detect the current platform.\n//...,public static Platform detect() throws Unsuppo...,java
2,// Gets the node meta data.\n// \n// @param ke...,public <T> T getNodeMetaData(Object key) {\n ...,java
3,// Copies all node meta data from the other no...,public void copyNodeMetaData(ASTNode other) {\...,java
4,// Sets the node meta data.\n// \n// @param ke...,"public void setNodeMetaData(Object key, Object...",java


In [11]:
df_train = df_train_java.append(df_train_go, ignore_index=True)
df_valid = df_valid_java.append(df_valid_go, ignore_index=True)
df_test = df_test_java.append(df_test_go, ignore_index=True)

<ipython-input-11-10c3ab1b794f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train_java.append(df_train_go, ignore_index=True)
<ipython-input-11-10c3ab1b794f>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_valid = df_valid_java.append(df_valid_go, ignore_index=True)
<ipython-input-11-10c3ab1b794f>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test_java.append(df_test_go, ignore_index=True)


In [12]:
df_train

,nl,code,language
0,// Compare the supplied plaintext password to ...,"public static boolean check(String passwd, Str...",java
1,// Attempt to detect the current platform.\n//...,public static Platform detect() throws Unsuppo...,java
2,// Gets the node meta data.\n// \n// @param ke...,public <T> T getNodeMetaData(Object key) {\n ...,java
3,// Copies all node meta data from the other no...,public void copyNodeMetaData(ASTNode other) {\...,java
4,// Sets the node meta data.\n// \n// @param ke...,"public void setNodeMetaData(Object key, Object...",java
...,...,...,...
332206,// SetInputConfigurations sets the InputConfig...,func (s *StartApplicationInput) SetInputConfig...,go
332207,// SetApplicationUpdate sets the ApplicationUp...,func (s *UpdateApplicationInput) SetApplicatio...,go
332208,// GetIdempotencyToken returns a randomly gene...,func GetIdempotencyToken() string {\n\tb := ma...,go
332209,// SetIdempotencyToken will set the value prov...,func SetIdempotencyToken(v reflect.Value) {\n\...,go


In [13]:
df_valid

,nl,code,language
0,// Copies the contents of this source to the g...,@CanIgnoreReturnValue\n public long copyTo(Ch...,java
1,// Reads the contents of this source as a stri...,public String read() throws IOException {\n ...,java
2,// Reads all the lines of this source as a lis...,public ImmutableList<String> readLines() throw...,java
3,// buffer when possible.,"@Override\n public int read(byte[] b, int off...",java
4,"// Returns a new CharBuffer identical to buf, ...",private static CharBuffer grow(CharBuffer buf)...,java
...,...,...,...
12503,// PublishNamedEvent which name infered from e...,"func PublishNamedEvent(pub Publisher, eventBod...",go
12504,// PublishEventContext publish event for given...,"func PublishEventContext(ctx context.Context, ...",go
12505,// PublishNamedEventContext publish named even...,func PublishNamedEventContext(ctx context.Cont...,go
12506,// String returns stirng representation of log...,func (level Level) String() string {\n\tswitch...,go


In [14]:
df_test

,nl,code,language
0,// Makes sure the fast-path emits in order.\n/...,protected final void fastPathOrderedEmit(U val...,java
1,// Wraps an ObservableSource into an Observabl...,@CheckReturnValue\n @SchedulerSupport(Sched...,java
2,// Returns an Observable that emits the events...,@CheckReturnValue\n @SchedulerSupport(Sched...,java
3,// Child Observers will observe the events of ...,public static <T> ConnectableObservable<T> obs...,java
4,// Creates an UnicastProcessor with the given ...,@CheckReturnValue\n @NonNull\n public st...,java
...,...,...,...
19072,// Error writes the given HTTP status to the c...,"func (r *Render) Error(status int, message ......",go
19073,// Renderer is a Middleware that maps a render...,func Renderer(options ...RenderOptions) Handle...,go
19074,//NewClient - constructor for a new dispenser ...,"func NewClient(apiKey string, url string, clie...",go
19075,//GetTask - wrapper to rest call to GET task f...,func (s *PDClient) GetTask(taskID string) (tas...,go


# Train

In [15]:
tokenizer_code = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer_nl = AutoTokenizer.from_pretrained(MODEL_NAME)

In [16]:
class Code2TextDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        code = self.df["language"].iloc[i] \
            + " docstring: " + self.df['code'].iloc[i]
        nl = self.df['nl'].iloc[i]

        code_tokenized = tokenizer_code(
            code,
            padding="max_length",
            max_length=64,
            truncation=True)
        code_input_ids = code_tokenized.input_ids
        code_attention = code_tokenized.attention_mask

        nl_tokenized = tokenizer_nl(
            nl,
            padding="max_length",
            max_length=64,
            truncation=True)
        nl_input_ids = nl_tokenized.input_ids

        return {
            "input_ids": code_input_ids,
            "labels": nl_input_ids,
            "attention_mask": code_attention
        }

In [17]:
# TODO: Implement this and add as parameter in Trainer
def compute_metrics(eval_pred):
    print(eval_pred)
    logits, labels = eval_pred
    return 1

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
#model.cuda()

training_args = TrainingArguments(
    output_dir="code2nl",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Code2TextDataset(df_train),
    eval_dataset=Code2TextDataset(df_valid),
)
trainer.train()

# Generate

In [ ]:
text = \
    'java docstring: void printMessage(String message) { System.out.println(message); }'

input_ids = tokenizer_code(text, return_tensors="pt").input_ids.cuda()
outputs = model.generate(input_ids)

print("\n//".join(tokenizer_nl.decode(outputs[0], skip_special_tokens=True)
    .split("//"))[1:])